In [1]:
!pip install openai

In [2]:
!pip install llama-index

In [10]:
!pip install transformers requests beautifulsoup4 


     |████████████████████████████████| 9.5 MB 197 kB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 624 kB/s eta 0:00:01
     |████████████████████████████████| 417 kB 582 kB/s eta 0:00:01
     |████████████████████████████████| 383 kB 370 kB/s eta 0:00:01
  Using cached filelock-3.15.4-py3-none-any.whl (16 kB)


In [11]:
!pip install pdfplumber

     |████████████████████████████████| 59 kB 631 kB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 1.9 MB/s eta 0:00:01


In [15]:
import os
import openai

In [16]:
os.environ['LLAMA_INDEX_CACHE_DIR'] = './cache'

In [17]:
# api_key = ""
# api_key = ""

In [18]:
openai.api_key =api_key

In [19]:
os.environ["OPENAI_API_KEY"] = api_key

## Download and clean data from URL

In [6]:
import logging
import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
from IPython.display import Markdown, display

In [20]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(temperature=0.2, model="gpt-4")

In [21]:
import fitz  # PyMuPDF

# Function to extract text from a two-column PDF
def extract_two_column_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        page_width, page_height = page.rect.width, page.rect.height
        
        # Define the coordinates for the left and right columns
        left_column_rect = fitz.Rect(0, 0, page_width / 2, page_height)
        right_column_rect = fitz.Rect(page_width / 2, 0, page_width, page_height)
        
        # Extract text from the left column
        left_column_text = page.get_text("text", clip=left_column_rect)
        
        # Extract text from the right column
        right_column_text = page.get_text("text", clip=right_column_rect)
        
        # Merge text in the reading order (left column first, then right column)
        full_page_text = left_column_text + "\n" + right_column_text
        
        # Print or process the text
        text += full_page_text
    return text

# Example usage
print(extract_two_column_text("./cache/document.pdf"))

VP Bank Ltd  ·  Valid
Gen
Terms and C

d from 1 April 2021
neral  
Conditions

Basic Terms and Conditions
	 1.	 Parties to the agreemen
4
	 2.	 Language
4
	 3.	 Service
4
	 4.	 Right of disposa
4
 5. Identity verificatio
5
	 6.	 Incapacity or limited capacit
5
 7. Notification of change
5
	 8.	 Execution of order
5
	 9.	 Errors in transmissio
6
	10.	 Recording of telephone conversations  
and electronic communication
6
	11.	 Communications from the Ban
6
	12.	 Dormant account
6
	13.	 Complaints and approva
6
	14.	 Conditions and cost
7
	15.	 Foreign currencies and foreign currency account
7
	16.	 Cheque
7
	17.	 Stock exchange transactions, trading  
and brokerag
7
	18.	 Transport, mailing and insuranc
7
	19.	 Safe custod
8
 20. Compliance with legislation / fiscal probity /  
economic sanction
11
	21.	 Release from banking secrecy / forwarding  
of dat
11
	22.	 Investment consulting and asset managemen
12
	23.	 Legal and tax advic
13
	24.	 Financial inducement
13
	25.	 Outsourcing

In [52]:
import requests
from bs4 import BeautifulSoup
from llama_index.core import VectorStoreIndex, Document
from collections import defaultdict
from llama_index.core import SimpleDirectoryReader
# Step 1: Fetch the content from the URL
url = "https://docs.ondo.finance/legal/terms-of-service"
response = requests.get(url)
if 'pdf' in url:
    # Step 2: Save the PDF content temporarily
    pdf_filename = "./cache/document.pdf"
    with open(pdf_filename, 'wb') as f:
        f.write(response.content)


    doc = extract_two_column_text(pdf_filename)

    # Step 4: Create a document for LlamaIndex
    document = Document(text=doc)
    # Step 5: Create an index from the document
    index = VectorStoreIndex.from_documents([document])
else:
    

    if response.status_code == 200:
        # Step 2: Parse HTML and extract text
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Step 3: Prepare to collect sections (headings + paragraphs)
        sections = defaultdict(str)  # Dictionary to store sections with headings as keys
        current_heading = None
        
        # Loop through the content and extract headings and paragraphs
        for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):  # Get both headings and paragraphs
            if element.name.startswith('h'):  # If it's a heading
                current_heading = element.get_text(strip=True)  # Use heading as the key
            elif element.name == 'p' and current_heading:  # If it's a paragraph under a heading
                sections[current_heading] += element.get_text(strip=True) + "\n"

        # Step 4: Prepare documents for each section
        documents = []
        for heading, content in sections.items():
            full_text = f"{heading}\n{content}"  # Combine heading and content into a single document
            documents.append(Document(text=full_text))
        
        # Step 5: Create an index from the documents
        index = VectorStoreIndex.from_documents(documents)



    else:
        print(f"Failed to retrieve the content from the URL. Status code: {response.status_code}")

In [53]:
# Step 6: Define the search queries
results = {"Location": [],
           "Explanation": "",
           "Raw": "",
           "isTrue": True}

queries = {
    "Location": """show the title and original text from document containing information about fees\nIf the documents does not contains any information about fees/costs output is: False\n\n
    If the documents have contents about the fees: shows Instruction for output in the form of: Title:original text of title\nContent: original text bellow title \n""",
}

In [54]:
location_response = index.as_query_engine().query(queries['Location'])
location_response.response

'Title: Fees and Price Estimates\nContent: In connection with your use of the Services, you are required to pay all fees necessary for interacting with the Ethereum or any other applicable blockchain, including transaction costs, as well as any other fees reflected on the Site at the time of your use of the Services. Where relevant, although we attempt to provide accurate fee information, this information reflects our estimates of fees, which may vary from the actual fees paid to use the Services and interact with the Ethereum or any other applicable blockchain.'

In [55]:
queries_sumary = f"Summarize the sections {location_response.response} that discuss fees or costs."
if 'False' not in location_response.response: 
    results['Location'] = []
    results['Location'].append(location_response.response.split("\n")[0].strip("Title: "))
    results["Raw"] = location_response.response.split("\n")[1].strip("Content: ")
    results['isTrue'] = True
    results["Explanation"] = index.as_query_engine().query(queries_sumary).response
else:
    results['isTrue'] = False
    results["Explanation"] = "-"
    results["Raw"] = "-"
    results['Location'] = [""]

In [56]:
results

{'Location': ['Fees and Price Estimates'],
 'Explanation': 'The sections titled "Fees and Price Estimates" and "Bridge Service Fees" discuss the costs associated with using the Services. Users are required to pay all necessary fees for interacting with Ethereum or any other applicable blockchain, including transaction costs and any other fees listed on the Site at the time of use. The fees are estimated and may vary from the actual fees paid. In addition, users may need to pay certain fees, such as gas fees, to execute smart contracts and use the Bridge Service.',
 'Raw': 'In connection with your use of the Services, you are required to pay all fees necessary for interacting with the Ethereum or any other applicable blockchain, including transaction costs, as well as any other fees reflected on the Site at the time of your use of the Services. Where relevant, although we attempt to provide accurate fee information, this information reflects our estimates of fees, which may vary from th